In [1]:
import os

# 현재 작업 디렉토리 가져오기
currentDirectory = os.getcwd()
print("현재 디렉토리:", currentDirectory) 

# 3개 위의 폴더 찾기
folder_path_HYU = os.path.abspath(os.path.join(currentDirectory, "../../.."))
folder_path_gradThesis_data = os.path.join(folder_path_HYU,'gradThesis', 'data')
print("데이터 경로:", folder_path_gradThesis_data)

import pandas as pd
import numpy as np
import time

import pickle

from LOL_data_pipeline import *
import UsefulModule

api_key = 'RGAPI-588d211a-21aa-4445-9b7b-3b46b38502ec'

API_hander = APIHander(api_key)

현재 디렉토리: c:\Users\erich\Desktop\HYU\응통인턴\2024 가을\프로젝트
데이터 경로: c:\Users\erich\Desktop\HYU\gradThesis\data


In [2]:
with open(f"{folder_path_gradThesis_data}//game_summaries_14_21.pkl", "rb") as file:  # "wb" 모드는 쓰기 및 바이너리 모드
    game_summaries = pickle.load(file)

print(f"{len(game_summaries)}")
# 모든 게임 저장 완료

game_minutes = np.array([len(g) for g in game_summaries])
game_minutes += 4 #벡터 연산

#기록된 게임의 시간과 각 게임이 몇개나 있는지 확인
recorded_minute, number_of_games = np.unique(game_minutes, return_counts=True)

10452


In [3]:
for i in range(len(recorded_minute)):
    print(f"{recorded_minute[i]} : {number_of_games[i]}")

16 : 2079
17 : 518
18 : 181
19 : 522
20 : 578
21 : 532
22 : 566
23 : 693
24 : 828
25 : 803
26 : 735
27 : 768
28 : 730
29 : 658
30 : 666
31 : 594
32 : 528
33 : 400
34 : 331
35 : 255
36 : 204
37 : 162
38 : 148
39 : 104
40 : 69


In [5]:
desired_column_order = ['blue_TOP_K','blue_TOP_D','blue_TOP_A','blue_TOP_CS',
                        'blue_JUG_K','blue_JUG_D','blue_JUG_A','blue_JUG_CS',
                        'blue_MID_K','blue_MID_D','blue_MID_A','blue_MID_CS',
                        'blue_ADC_K','blue_ADC_D','blue_ADC_A','blue_ADC_CS',
                        'blue_SUP_K','blue_SUP_D','blue_SUP_A','blue_SUP_CS',
                        'red_TOP_K','red_TOP_D','red_TOP_A','red_TOP_CS',
                        'red_JUG_K','red_JUG_D','red_JUG_A','red_JUG_CS',
                        'red_MID_K','red_MID_D','red_MID_A','red_MID_CS',
                        'red_ADC_K','red_ADC_D','red_ADC_A','red_ADC_CS',
                        'red_SUP_K','red_SUP_D','red_SUP_A','red_SUP_CS',
                        'blue_TOP_TURRET','blue_TOP_INHIBITOR',
                        'blue_MID_TURRET','blue_MID_INHIBITOR',
                        'blue_BOT_TURRET','blue_BOT_INHIBITOR',
                        'red_TOP_TURRET','red_TOP_INHIBITOR',
                        'red_MID_TURRET','red_MID_INHIBITOR',
                        'red_BOT_TURRET','red_BOT_INHIBITOR',
                        'blue_NEXUS_TURRET','red_NEXUS_TURRET',
                        'blue_TOP_TURRET_PLATE','blue_MID_TURRET_PLATE','blue_BOT_TURRET_PLATE',
                        'red_TOP_TURRET_PLATE','red_MID_TURRET_PLATE','red_BOT_TURRET_PLATE',
                        'blue_HORDE',
                        'blue_DRAGON',
                        'blue_ELDER_DRAGON',
                        'blue_RIFTHERALD',
                        'blue_BARON_NASHOR',
                        'red_HORDE',
                        'red_DRAGON',
                        'red_ELDER_DRAGON',
                        'red_RIFTHERALD',
                        'red_BARON_NASHOR',
                        'blue_TOP_LEVEL',
                        'blue_JUG_LEVEL',
                        'blue_MID_LEVEL',
                        'blue_ADC_LEVEL',
                        'blue_SUP_LEVEL',
                        'red_TOP_LEVEL',
                        'red_JUG_LEVEL',
                        'red_MID_LEVEL',
                        'red_ADC_LEVEL',
                        'red_SUP_LEVEL',
                        'blue_win']

In [6]:
n = 19

game_end_target = game_minutes == n
# 원하는 시간대에 종료된 게임만 불러옴

game_summaries_target = [game_summaries[i] for i in range(len(game_summaries)) if game_end_target[i]]
# game_summaries_target[gameIndex][minuteIndex]

# len(game_summaries_target) 

train, test = UsefulModule.list_train_test_split(game_summaries_target, 0.2, 2019089561)

train_set = pd.DataFrame()

k = 0
for gameIndex in range(len(train)):
    train_set = pd.concat([train_set, train[gameIndex][k]])

train_set = train_set[desired_column_order]
# column 순서 한 번 맞춰주고

# blue_win이 y니까 X, y 분류
train_set_X = train_set.drop(columns=["blue_win"])
train_set_y = train_set.loc[:, "blue_win"]

In [9]:
type(train_set_y)

pandas.core.series.Series

# Random Forest 적합

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [6]:
rf_models = [[] for _ in range(16,41)]
# rf_models 리스트는 첫 인덱스를 게임 종료 시간(n), 그리고 두번째 인덱스를 게임 내 시간(k)로 함.
accuracies = [[] for _ in range(16,41)]
# accuracies도 마찬가지, 첫 인덱스를 게임 종료 시간(n), 그리고 두번째 인덱스를 게임 내 시간(k)로 함.

for idx, n in enumerate(range(16,41)):
    game_end_target = game_minutes == n
    # 원하는 시간대에 종료된 게임만 불러옴
    game_summaries_target = [game_summaries[i] for i in range(len(game_summaries)) if game_end_target[i]]
    # game_summaries_target[gameIndex][minuteIndex]

    # train, test 나눠주고
    train, test = UsefulModule.list_train_test_split(game_summaries_target, 0.2, 2019089561)

    # 각 게임을 돌면서

    # k : 현재 게임 시간(5,6,7, ... 분)
    for k in range(len(train[0])):
        st = time.time() #모델 적합 시작 시간
        print(f"Model Fitting, n : {n}, k : {k}", end = " ")

        # 특정 게임 시간에 대해서 train, test set을 만들어야하기 때문에 여기서 초기화
        train_set = pd.DataFrame()

        # gameIndex : 게임 번호
        for gameIndex in range(len(train)):
            train_set = pd.concat([train_set, train[gameIndex][k]])
        # 여기까지 완료되면, n-1 ~ n 분 사이에 종료된 게임에 대해서,
        # k분 까지 정보를 준 train set이 완성됨. 여기서 이제 X, y split을 해야함.

        train_set = train_set[desired_column_order]
        # column 순서 한 번 맞춰주고

        # blue_win이 y니까 X, y 분류
        train_set_X = train_set.drop(columns=["blue_win"])
        train_set_y = train_set.loc[:, "blue_win"]

        print(f"{len(list(train_set_X.columns))}, {len(train_set_X)}, {len(train_set_y)}", end = " ")

        # test set에 대해서도 똑같이 적용
        test_set = pd.DataFrame()
        
        # gameIndex : 게임 번호
        for gameIndex in range(len(test)):
            test_set = pd.concat([test_set, test[gameIndex][k]])
        # 여기까지 완료되면, n-1 ~ n 분 사이에 종료된 게임에 대해서,
        # k분 까지 정보를 준 train set이 완성됨. 여기서 이제 X, y split을 해야함.

        test_set = test_set[desired_column_order]
        # column 순서 한 번 맞춰주고

        # blue_win이 y니까 X, y 분류
        test_set_X = test_set.drop(columns=["blue_win"])
        test_set_y = test_set.loc[:, "blue_win"]

        print(f"{len(list(test_set_X.columns))}, {len(test_set_X)}, {len(test_set_y)}", end = " ")

        rf_model = RandomForestClassifier(n_estimators=100, random_state=2019089561)
        rf_model.fit(train_set_X, train_set_y)
    
        y_pred = rf_model.predict(test_set_X)
        
        et = time.time()
        print(f", elapsed time : {round(et-st,4)}", end = " ")
        print(f", {accuracy_score(test_set_y, y_pred)}")

        rf_models[idx].append(rf_model)
        accuracies[idx].append(accuracy_score(test_set_y, y_pred))


Model Fitting, n : 16, k : 0 80, 1296, 1296 80, 323, 323 , elapsed time : 1.5819 , 0.8544891640866873
Model Fitting, n : 16, k : 1 80, 1296, 1296 80, 323, 323 , elapsed time : 1.5919 , 0.8792569659442725
Model Fitting, n : 16, k : 2 80, 1296, 1296 80, 323, 323 , elapsed time : 1.5957 , 0.9040247678018576
Model Fitting, n : 16, k : 3 80, 1296, 1296 80, 323, 323 , elapsed time : 1.6469 , 0.9535603715170279
Model Fitting, n : 16, k : 4 80, 1296, 1296 80, 323, 323 , elapsed time : 1.6146 , 0.9659442724458205
Model Fitting, n : 16, k : 5 80, 1296, 1296 80, 323, 323 , elapsed time : 1.6203 , 0.9690402476780186
Model Fitting, n : 16, k : 6 80, 1296, 1296 80, 323, 323 , elapsed time : 7.9851 , 0.9845201238390093
Model Fitting, n : 16, k : 7 80, 1296, 1296 80, 323, 323 , elapsed time : 3.1082 , 0.9845201238390093
Model Fitting, n : 16, k : 8 80, 1296, 1296 80, 323, 323 , elapsed time : 3.0823 , 0.9876160990712074
Model Fitting, n : 16, k : 9 80, 1296, 1296 80, 323, 323 , elapsed time : 3.0086 ,

In [7]:
with open(f"{folder_path_gradThesis_data}//rf_models_14_21.pkl", "wb") as file:  # "wb" 모드는 쓰기 및 바이너리 모드
    pickle.dump(rf_models, file)
with open(f"{folder_path_gradThesis_data}//accuracies_14_21.pkl", "wb") as file:  # "wb" 모드는 쓰기 및 바이너리 모드
    pickle.dump(accuracies, file)

In [2]:
with open(f"{folder_path_gradThesis_data}//rf_models_14_21.pkl", "rb") as file:  # "wb" 모드는 쓰기 및 바이너리 모드
    rf_models = pickle.load(file)
with open(f"{folder_path_gradThesis_data}//accuracies_14_21.pkl", "rb") as file:  # "wb" 모드는 쓰기 및 바이너리 모드
    accuracies = pickle.load(file)

print(f"{len(rf_models)}")
print(f"{len(accuracies)}")

25
25


# 시각화